In [11]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as im
from sklearn.cluster import KMeans
import os
import imageio
from PIL import Image

%matplotlib inline


In [12]:
def Initialization (foldername,topcolors):

    #Clothing part
    #foldername = "./fashion";
    #topcolors = 2 #the number of top colors we want to compare

    #iterate through folder and find kmeans data

    num_points = len(os.listdir(foldername))

    centerPoints = np.zeros((num_points,topcolors,3))
    weights = np.ones((topcolors,1))
    count =0

    indiceList = []
    for item in os.listdir(foldername):
 
        image = imageio.imread(foldername+'/'+item)
    
        x = image.shape[0]
        y = image.shape[1]
        z = image.shape[2]
        
        indiceList.append([item])

    
        newim = np.reshape(image,(x*y,z))

    
        kmeans = KMeans(n_clusters=topcolors, random_state=0).fit(newim)
        centers = kmeans.cluster_centers_  #cluster centers
        cluster_ids = kmeans.labels_
        for i in range(0,topcolors):
            vec = (cluster_ids == i)
            num = np.sum(vec)
            weights[i] = num/(x*y)
        
        order = np.argsort(weights,axis = None)
        
        centerPoints[count,:,:]=centers[order] #the top colors in the picture
        
        count = count+1
        
    return centerPoints,indiceList
    
    
    
    
    
    
    



In [13]:
def Euclid_dist(x, y):

    ndim = x.shape[0];

    dist= np.sqrt(np.sum(np.square(x-y)))

        
    return dist
    raise NotImplementedError
    



def distanceCalc(centerPoints):
    #centePoints is num*col*3
    num = centerPoints.shape[0]
    col = centerPoints.shape[1]
    FirstIm = np.zeros((3,1));
    SecondIm = np.zeros((3,1));
    distvec = np.zeros((num,num))
    dist = 0
    
    for i in range(0,num):
        for j in range(0,num):
            for k in range(0,col):
                FirstIm[0] = centerPoints[i,k,0];
                FirstIm[1] = centerPoints[i,k,1];
                FirstIm[2] = centerPoints[i,k,2];
                
                SecondIm[0] = centerPoints[j,k,0]
                SecondIm[1] = centerPoints[j,k,1]
                SecondIm[2] = centerPoints[j,k,2]
                
                dist = dist + Euclid_dist(FirstIm,SecondIm)
                
                
            distvec[i,j] = dist
            distvec[j,i] = dist
            dist = 0
            
            
            
    return distvec




def findGroup(WantedDist,distances):
    indices = np.argwhere(distances <= WantedDist)
    
    num = indices.shape[0]
    newvec = []
    
    for i in range(0,num):
        for j in range(0,num):
            if(indices[j,0] == i): #if first indice = i, then add second indices to newvec
                if (~np.any(newvec == indices[j,1])):
                    newvec.append(indices[j,1])
            
        if (newvec[-1]!=-1):
            newvec.append(-1)
    
    return newvec
    
    """
    for i in range(0,num):
       
        for j in range(0,num):
            if (indices[j,0] == i):
                if (count == 0):
                    newvec = indices[j,1]
                else:
                    newvec = np.append(newvec,indices[j,1])
                count = count+1
        count = 0;
        
        if(i == 0):
            store = newvec
        else:
            store = np.append(store,newvec)
            
        store = np.append(store,-1)
        """
        
    #return store
        
      
    
def Clustering(foldername, topcolors,layers,maxClusternum):
    centerPoints, indiceList = Initialization(foldername,topcolors)
    num = centerPoints.shape[0]
    col = centerPoints.shape[1]
    
    distmat = distanceCalc(centerPoints)
    minDist = np.min(distmat)
    maxDist = np.max(distmat)
    
    
    count = 0
    totSum = np.zeros((1,col,3))
    newClusterNum = 0;
    
    tempCenterPoints = np.zeros((num,col,3))
    prevstore = findGroup(minDist,distmat)
    
    output = []
    
    tempList = []
    tempindiceList = []
    
    
    for radius in np.linspace(minDist,maxDist,layers):
        store =findGroup(radius,distmat)
        
        if(~np.array_equal(store ,prevstore)):
            for indices in store:
                if(indices != -1):
                    count = count+1
                    totSum = totSum + centerPoints[indices,:,:]
                    for item in indiceList[indices]:
                        
                        tempList.append(item)
                else:
                    compVec = np.zeros(totSum.shape)

                    if (~np.array_equal(compVec,totSum)):
                        tempCenterPoints[newClusterNum,:,:] = totSum/count
                        newClusterNum = newClusterNum + 1
                        tempindiceList.append(tempList)

                        
                    else:
                        newClusterNum = 0
                        
                    tempList=[]
                        
                    
                        
                    count = 0
                    totSum = np.zeros((1,col,3))
                    
                    
                    
            centerPoints = tempCenterPoints[0:newClusterNum,:,:]
            
            tempCenterPoints = np.zeros((num,col,3))
            distmat = distanceCalc(centerPoints)
            output.append(store)
            
            prevstore = store
            indiceList= tempindiceList
            tempindiceList = []
            #print("This is the cluser num")
            #print(newClusterNum)
            if(newClusterNum <= maxClusternum):
                print("Im here")
                return indiceList
            
            newClusterNum = 0
            
        
        
            
            
            
                    
    return indiceList
                    
                    
    

                

In [14]:
def fileWrite(outputData,foldername):
    picNums = os.listdir(foldername)
    count = 0
    #string = "Cluster "+str(0)+","
    string = "Cluster \n"
    clusterNum = len(outputData)
    
    for i in range(0,clusterNum):
        string = string + str(count) +","
        for j in outputData[i]:
            string = string+j[0:-4]+","
                
        string = string + "\n"
        #string = string+"Cluster "+str(count)+","
        
        count = count+1
            
    file = open("OutputData.csv","w") 
    file.write(string)
    file.close()
    
    print("fileCreated")
        
    

In [ ]:
foldername = "./fashion"
outputData = Clustering(foldername,4,100,7)



In [ ]:
fileWrite(outputData,foldername)

In [48]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import numpy as np
from ete3 import Tree, TreeStyle ,ImgFace, faces, NodeStyle
from PIL import Image
import os
import csv

OutputData = pd.read_csv('C:\\Users\\Rachvan23\\Desktop\\ClothingClustering-master\\ClothingClustering-master\\OutputData.csv')
fashion = ('C:\\Users\\Rachvan23\\Desktop\ClothingClustering-master\\ClothingClustering-master\\fashion\\')
pics = os.listdir(fashion)
OutputData = OutputData.fillna(" ")

x = np.array([np.array(Image.open(fashion+fname)) for fname in pics])


t = Tree()
t.populate(7, ["Cluster0", "Cluster1", "Cluster2", "Cluster3", "Cluster4", "Cluster5", "Cluster6"])
style = NodeStyle()
style["vt_line_width"] = 20
style["hz_line_width"] = 10
t.set_style(style)




"""for leaf in t.iter_leaves():
    leaf.img_style["size"] = 150
    for i in range(0,7):
        for j in range(1, OutputData.shape[1]):
            print(OutputData.iloc[i][j])
            newImg = faces.ImgFace(fashion+OutputData.iloc[i][j])
            t.add_face(newImg, column = j)
            leaf.add_face(newImg, j)"""

def mylayout(node):
    if len(node)>=4:
        col = 0
        for i, name in enumerate(set(node.get_leaf_names())):
            if i > 0 and i % 2 == 0:
                col += 1
                for j in range(1, OutputData.shape[1]):
                    if name == ("Cluster" + str(i)):
                        newFace = faces.ImgFace(fashion+OutputData.iloc[i][j], width = 50)
                        faces.add_face_to_node(newFace, node, column = col)

                    

            
ts = TreeStyle()
ts.layout_fn = mylayout
t.show()
